In [ ]:
# Source: https://huggingface.co/learn/nlp-course/en/chapter7/5?fw=pt

In [1]:
!pip install transformers==4.30
!pip install accelerate -U
!pip install rouge_score
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 27.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.30.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0

In [2]:
import pandas as pd
import torch
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
import numpy as np
import nltk
import evaluate
rouge_score = evaluate.load("rouge")
nltk.download("punkt")
from transformers import DataCollatorForSeq2Seq
from nltk.tokenize import sent_tokenize
import csv

2024-04-25 17:25:25.763483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 17:25:25.763585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 17:25:25.885893: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:

test_data = Dataset.from_pandas(pd.read_csv("/kaggle/input/tfidfdatatest/hindi_test_final_TFIDF.csv")[["Article", "Summary"]].dropna())

In [5]:
# Loading our fine-tuned models
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/tfidfmodel")
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/tfidfmodel").to(device)

In [6]:
def get_summary(article):
    temp = tokenizer(article, truncation = True, padding=True, return_tensors='pt', max_length=1024).input_ids.to(device)
    out = model.generate(temp, max_length=150, num_beams=5, length_penalty=1.2, early_stopping=True) 
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [7]:
print(get_summary(test_data["Article"][0]))

उल्कापिंड जब धरती पर गिरते हैं, तो इनकी चमक इतनी ज्यादा होती है कि लगभग 200 से 300 किलोमीटर के दायरे के लोग आसमान में इसे देख सकते हैं। हालांकि इस घटना ने सबको आश्चर्य में डाल दिया है। दरअसल, उल्कापिंड जब धरती पर गिरते हैं, तो इनकी चमक इतनी ज्यादा होती है कि लगभग 200 से 300 किलोमीटर के दायरे के लोग आसमान में इसे देख सकते हैं।


In [8]:
test_summaries = []
for i in test_data["Article"]:
    temp_lst = []
    temp_lst.append(i)
    temp_lst.append(get_summary(i))
    test_summaries.append(temp_lst)
with open("/kaggle/working/test_tfidf.csv", "w") as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(["Article", "Generated Summary"])
    csvwriter.writerows(test_summaries)